# AB测绘实战学习
- 掌握如何设计AB测试
- AB测试效果如何解读
- 账务如何进行实验设计与效果计算

# 1、什么是AB测试？
AB测试是通过对比不同版本的特定指标，从而指导后续的决策和优化，简言之“以对照只优劣”

# 2、AB测试关键点是什么？
指定目标和KPI策略（策略可以从差异点角度分析）

# 3、AB测试的作用是什么？
两点：
- 实现KPI的最大化
- 后续的分析、沉淀诀窍

# 4、AB测试实施的一般步骤有哪些？
- 目标人群分组
- 数据结果回收
- 计算实验效果
- 效果对比得出最有策略
- 分析策略
- 设计下一轮实验

# 6、AB测试如何分配流量？

两种方法：
- 随机数排序
- 利用某些随机数ID尾数

`确定最小样本数量URL：` https://www.evanmiller.org/ab-testing/sample-size.html

# 7、为什么要计算参与AB测试的最少参与人数？
首先AB测试目的是不改变用户体验前提下，让少部分抽样用户来验证方案优劣
- 抽取用户过少，不能代表所有用户观点，结果没有意义
- 抽取用户过多，一旦薪方案与预期效果偏差较大则会对用户体验带来较大的影响

# 8、什么是辛普森悖论？

![辛普森悖论](./img/辛普森悖论.png)

辛普森悖论是英国统计学家E·H·辛普森有951年提出，在某个条件下的两组数据，分别讨论时都会满足某种行止，一旦合并考虑，却可能导致相反的结论。

# 9、AB测试中如何避免辛普森悖论？
- 在AB测试中必须进行合理的正确流量分割，保证实验组和对照组中用户特征一直，并且具有代表性，可以代表总体用户特征。
- 实验设计中，某些标亮对实验结果有影响就需要对这两个变量在同层做互斥实验
- 实验设计中，需要积极的进行多维度的细分分析，除总体对比也需要看细分受众群体实验结果，不要以偏概全，也不要以全盖偏

# 小结
AB测试的应用场景：
- 互联网行业应用广泛：页面结构调整、换新图标、添加新功能
- 实体行业应用相对复杂一些，不同优惠券效果测试
AB测试还是ABC……测试
- AB测试：一次测试一方案
- ABC……测试：一次测试多个方案，但是需要流量足够大，否则难以满足实验要求的最小人数
AB测试需要注意那些点：
- 流量分配
- 确定有效的最小参与人数
    - 确定基准指标和提升目标
    - 设置显著水平α （一般是5%）和统计功效1-β（一般是80%）
    - 出结果之后计算P值，如果P<5%，那么可以拒绝原假设
- 可以通过AB测试工具网站确定人数，也可以使用Python工具statsmodels模块代码来实现
    - import statsmodels.stats.api 计算需要人数
    - statsmodels.stats.proportion 计算P值和置信区间

In [3]:
%%sql
select a.*,case when rand()<0.1 then 'ctrl'
when rand() between 0.1 and 0.55 then 'test1' else 'test2' end as ab_group_tag
from (select distinct customerID from user_table) a
order by ab_group_tag;

In [ ]:
%%sql
select a.*,case when customerID like '1%' then 'ctrl'
when customerID like '2%' or customerID like '3%' or customerID like '4%' or customerID like '5%' then 'test1'
else 'test2' end as ab_group_tag
from(select distinct customerID from user_table) a 
order by ab_group_tag;

In [2]:
# 导入numpy包
import numpy as np
# 导入pandas包
import pandas as pd
# 导入统计包
import scipy.stats as stats
# 导入统计和计量经济学包
import statsmodels.stats.api as sms
# 导入matplotlib2D-pyplot绘图库
import matplotlib.pyplot as plt
# 导入matplotlib2D绘图库
import matplotlib as mpl
# 导入seaborn绘图库
import seaborn as sns
# 导入数学函数
from math import ceil

%matplotlib inline

# 计算effect_size 
'''
description: 计算两个比例之间的效果量
params：
    - prop1：假设实验组的概率
    - prop2：对照实验组大概率
'''
effect_size = sms.proportion_effectsize(0.13, 0.15)

'''
description: 求解对于任意一个参数的两个样本的Z测试幂
params：
    - effect_size：影响大小
    - nobs1：
    - alpha：
    - power：
    - ratio：
    - alternative：
'''
temp = sms.NormalIndPower()

required_n = sms.NormalIndPower().solve_power(
    effect_size, # 传入上面计算的 effect_size
    power=0.8, # 设置 1-β = 80%
    alpha=0.05, # 设置 α 为5%
    ratio=1  # 对照组和测试组人一样, 这里的ratio 比例就是1
)

#对结果向上取整
required_n = ceil(required_n)

print(required_n)

In [9]:
df = pd.read_csv('./file/ab_data.csv')

'''
user_id - 访问的用户ID
timestamp - 访问的时间
group - 该用户被放到那一组 {control对照, treatment实验}
landing_page -该用户看到的是哪一种落地页 {old_page老页面, new_page新页面}
converted - 改次访问是否有转化 (binary, 0=无转化, 1=转化)
'''
df.info()

In [10]:
# 构建透视表
df.pivot_table(index = 'group',columns='landing_page',values = 'user_id',aggfunc='count')

In [12]:
# 在我们进行后续处理之前, 还要查看是否有用户进行了多次操作
session_counts = df['user_id'].value_counts(ascending=False)
multi_users = session_counts[session_counts>1].count()
multi_users

In [13]:
# 说明一共有3894个用户访问了不止一次, 整体数据有20多万条, 所以我们直接把这部分数据删除
users = session_counts[session_counts < 2].index
df = df[df['user_id'].isin(users)]

In [16]:
control_sample = df[df.group == 'control'].sample(n = required_n, random_state = 0)
treatment_sample = df[df.group == 'treatment'].sample(n = required_n, random_state = 22)

ab_test = pd.concat([control_sample, treatment_sample], axis = 0)
ab_test.reset_index(drop = True, inplace = True)
ab_test

In [19]:
'''
landing_page:落地页类型
old_page:老页面
new_page:新页面
control:控制组
treatment:对照组
'''
ab_test.groupby('group')['landing_page'].value_counts()

In [26]:
conversion_rates = ab_test.groupby('group')['converted'].mean().to_frame()
conversion_rates

conversion_rates.style.format('{:.3f}')

In [48]:
# 3.5、假设检验

# 导入比例检验的proportions_ztest函数、计算比例的置信区间proportion_confint函数
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control_results = ab_test[ab_test.group == 'control']['converted']
treatment_results = ab_test[ab_test.group == 'treatment']['converted']

n_con = control_results.count()
n_treat = treatment_results.count()

successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

# 计算P值
z_stat, pval = proportions_ztest(successes, nobs = nobs)
# 计算置信区间
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs = nobs, alpha = 0.05)

print(f'z statistic ：{z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for control group:[{lower_treat:.3f}, {upper_treat:.3f}]')